In [ ]:
!pip3 -q install chromadb langchain tiktoken
!pip3 install langchain-community
!pip3 install huggingface
!pip3 install sentence-transformers
!pip3 install keras==2.11.0
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles
!pip install transformers accelerate einops --upgrade
!pip install langchain_community

In [ ]:
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader,TextLoader

In [ ]:
loader = DirectoryLoader("/content/new_articles",glob="./*.txt",loader_cls=TextLoader)
document=loader.load()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM ,BitsAndBytesConfig,pipeline
from langchain.llms import HuggingFacePipeline
import torch

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32
).to("cuda")

# Create the text-generation pipeline
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=250,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1
)

# Wrap the pipeline for LangChain
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    length_function=len  # this should return an integer
)


In [ ]:
text=splitter.split_documents(document)

In [ ]:
len(text)

**Creating db**

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
persist_directory="db"
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vector_db = Chroma.from_documents(
    documents=text,
    embedding=embedding_model,
    persist_directory=persist_directory
)

In [ ]:
vector_db.persist()
vector_db=None

In [ ]:
#to load the data from persist directory

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding_model
)

In [ ]:
vectordb.similarity_search("AI",k=2)

In [ ]:
#initialize a retriever

retriever = vectordb.as_retriever(search_kwargs={"k":4})


In [ ]:
docs=retriever.get_relevant_documents("hello there tell me about genetative ai")

In [ ]:
len(docs)

In [ ]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True
)

In [ ]:
query="what is the news about pando"
llm_response = qa_chain(query)
llm_response["result"][164::]

In [ ]:
len("Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n")

In [ ]:
llm.generate(prompts=["hello llm , how are you"])